In [18]:
#%pip install -r requirements.txt

In [2]:
import sys
from pathlib import Path

# Adiciona o diretório raiz do projeto no sys.path
root_path = Path().resolve()
if str(root_path) not in sys.path:
    sys.path.append(str(root_path))

In [189]:
import pandas as pd
import os
import yfinance as yf
import fundamentus as fd

2025-07-24 11:19:30,831 [logging.log_init] INFO: LOGLEVEL=INFO


In [4]:
dir_dados = 'dados'
path_planilha_b3_fii = os.path.join(dir_dados, 'movimentacao-fii-2025-07-23-10-29-05.xlsx')
path_planilha_b3_tudo = os.path.join(dir_dados, 'movimentacao-2025-07-23-14-40-14.xlsx') 
path_fusoes = os.path.join(dir_dados, 'fusoes_desdobramentos.csv')


In [6]:
from src.preprocessamento import ( carregar_movimentacoes, carregar_fusoes_desdobramentos, ajustar_movimentacoes_por_eventos, parse_valor_br )

In [132]:
df_fusoes = pd.read_csv(path_fusoes)
df_fusoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ativo_antigo     5 non-null      object
 1   ativo_novo       5 non-null      object
 2   tipo_evento      5 non-null      object
 3   fator_conversao  5 non-null      object
 4   data_vigencia    5 non-null      object
 5   fonte_oficial    5 non-null      object
dtypes: object(6)
memory usage: 372.0+ bytes


# Testando a carga de uma planilha apenas com FII

In [151]:
df_fii = pd.read_excel(path_planilha_b3_fii, dtype=str)

df_fii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Entrada/Saída      60 non-null     object
 1   Data               60 non-null     object
 2   Movimentação       60 non-null     object
 3   Produto            60 non-null     object
 4   Instituição        60 non-null     object
 5   Quantidade         60 non-null     object
 6   Preço unitário     60 non-null     object
 7   Valor da Operação  60 non-null     object
dtypes: object(8)
memory usage: 3.9+ KB


c:\Projetos\pynvest\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [152]:
df_fii['Produto'].unique()

array(['MXRF11 - MAXI RENDA FDO INV IMOB - FII',
       'BCFF11 - FII - FII BTG PACTUAL FUNDO DE FUNDOS RESPONS LTDA',
       'VGHF11 - VALORA HEDGE FUND FUNDO DE INVESTIMENTO IMOBILIARIO - FII',
       'XPML11 - XP MALLS FDO INV IMOB FII',
       'VISC11 - VINCI SHOPPING CENTERS FDO INVEST IMOB - FII',
       'BCFF11 - FDO INV IMOB - FII BTG PACTUAL FUNDO DE FUNDOS',
       'MXRF11 - MAXI RENDA FUNDO DE INVESTIMENTO IMOBILIARIO - FII',
       'RBVA11 - FDO INV IMOB RIO BRAVO RENDA VAREJO - FII',
       'XPLG11 - XP LOG FUNDO DE INVESTIMENTO IMOBILIARIO FII',
       'BTLG11 - BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IMOBILIARIO',
       'HGLG11 - CSHG LOGÍSTICA FDO INV IMOB - FII',
       'KNRI11 - KINEA RENDA IMOBILIÁRIA FDO INV IMOB - FII',
       'HCTR11 - HECTARE CE - FDO INV IMOB',
       'HCTR11 - HECTARE CE - FUNDO DE INVESTIMENTO IMOBILIÁRIO'],
      dtype=object)

In [101]:

def classificar_ativo_b3(row):
    ticker = str(row['Ticker']).upper()
    descricao = str(row['Descrição']).upper()

    # Fundos Imobiliários (FII) - mais variações comuns na descrição
    if ticker.endswith('11') and any(
        x in descricao for x in ['FII', 'FDO INV IMOB', 'FUNDO DE INVESTIMENTO IMOB', 'IMOBILIÁRIO']):
        return 'FII'

    # ETFs
    if ticker.endswith('11') and any(x in descricao for x in ['ETF', 'ÍNDICE']):
        return 'ETF'

    # BDRs
    if ticker.endswith(('34', '35', '32', '39')) or 'BDR' in descricao:
        return 'BDR'

    # Ações ON/PN
    if ticker.endswith('3'):
        return 'Ação ON'
    if ticker.endswith('4'):
        return 'Ação PN'

    # Units
    if ticker.endswith('11') and any(x in descricao for x in ['UNIT', 'UNITS']):
        return 'Unit'

    # Opções
    if len(ticker) >= 5 and ticker[-1].isdigit() and ticker[-2].isalpha():
        return 'Opção'

    # Renda Fixa Pública
    if any(p in descricao for p in ['TESOURO', 'LTN', 'NTN', 'LFT']):
        return 'Tesouro Direto'

    # Renda Fixa Privada
    if any(p in descricao for p in ['DEBENTURE', 'CRI', 'CRA', 'CDB']):
        return 'Renda Fixa Privada'

    # Fallback
    return 'Outro'

In [102]:
def extrair_e_classificar_ticker(df):
    # Extrai ticker e descrição
    if 'Produto' in df.columns:
        df[['Ticker', 'Descrição']] = df['Produto'].str.split(' - ', n=1, expand=True)
        df['Ticker'] = df['Ticker'].str.strip().str.upper()
        df['Descrição'] = df['Descrição'].str.strip().str.upper()
        pos = df.columns.get_loc('Produto')
        df.drop('Produto', axis=1, inplace=True)
        df.insert(pos, 'Descrição', df.pop('Descrição'))
        df.insert(pos, 'Ticker', df.pop('Ticker'))

    # Classificação única
    df_class = df.groupby('Ticker').first().reset_index()
    df_class['Tipo de Ativo'] = df_class.apply(classificar_ativo_b3, axis=1)

    # Merge final
    df = df.merge(df_class[['Ticker', 'Tipo de Ativo']], on='Ticker', how='left')
    return df

In [153]:
def preprocessar_dados_b3(df: pd.DataFrame) -> pd.DataFrame:
    colunas_monetarias = ['Valor da Operação', 'Preço unitário','Quantidade']
    if all(col in df.columns for col in colunas_monetarias):
        for col in colunas_monetarias:
            # Remove só pontos que são separadores de milhar (antes da vírgula)
            df[col] = df[col].str.strip().str.replace('\xa0', '')
            df[col] = df[col].str.replace(r'\.(?=\d{3}(?:\.|,|$))', '', regex=True)  # remove pontos de milhar
            df[col] = df[col].str.replace(',', '.')  # substitui vírgula decimal para ponto decimal
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    df = extrair_e_classificar_ticker(df)

    colunas_inferencia_tipo_ativo = ['Ticker', 'Descrição']
    if all(col in df.columns for col in colunas_inferencia_tipo_ativo):
        df['Tipo de Ativo'] = df.apply(classificar_ativo_b3, axis=1)

    # Define quais tipos são considerados compra ou venda
    tipos_compra_venda = [
        'Compra',
        'Venda',
        'Transferência - Liquidação',
        # esses dois aqui abaixo não aparecem, quando filtramos por "Compra/Venda"
        # ao puxar um extrato de movimentações na B3
        # 'Leilão de Fração',
        # 'Resgate',
    ]

    df['Compra/Venda'] = df['Movimentação'].isin(tipos_compra_venda)

    df.info()
    return df

In [154]:
df_fii = preprocessar_dados_b3(df_fii)
df_fii = df_fii.sort_values(by=['Ticker'], ascending=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Entrada/Saída      60 non-null     object 
 1   Data               60 non-null     object 
 2   Movimentação       60 non-null     object 
 3   Ticker             60 non-null     object 
 4   Descrição          60 non-null     object 
 5   Instituição        60 non-null     object 
 6   Quantidade         60 non-null     int64  
 7   Preço unitário     60 non-null     float64
 8   Valor da Operação  60 non-null     float64
 9   Tipo de Ativo      60 non-null     object 
 10  Compra/Venda       60 non-null     bool   
dtypes: bool(1), float64(2), int64(1), object(7)
memory usage: 4.9+ KB


## Validando o resultado

In [179]:
df_fii[df_fii['Ticker'].str.endswith('11')]['Ticker'].unique()

array(['BCFF11', 'BTLG11', 'HCTR11', 'HGLG11', 'KNRI11', 'MXRF11',
       'RBVA11', 'VGHF11', 'VISC11', 'XPLG11', 'XPML11'], dtype=object)

In [155]:
df_fii[['Ticker','Descrição','Tipo de Ativo']].drop_duplicates()


,Ticker,Descrição,Tipo de Ativo
10,BCFF11,FDO INV IMOB - FII BTG PACTUAL FUNDO DE FUNDOS,FII
5,BCFF11,FII - FII BTG PACTUAL FUNDO DE FUNDOS RESPONS ...,FII
33,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,FII
48,HCTR11,HECTARE CE - FDO INV IMOB,FII
53,HCTR11,HECTARE CE - FUNDO DE INVESTIMENTO IMOBILIÁRIO,FII
37,HGLG11,CSHG LOGÍSTICA FDO INV IMOB - FII,FII
42,KNRI11,KINEA RENDA IMOBILIÁRIA FDO INV IMOB - FII,FII
31,MXRF11,MAXI RENDA FUNDO DE INVESTIMENTO IMOBILIARIO -...,FII
0,MXRF11,MAXI RENDA FDO INV IMOB - FII,FII
24,RBVA11,FDO INV IMOB RIO BRAVO RENDA VAREJO - FII,FII


In [156]:
df_fii['Ticker'].unique()

array(['BCFF11', 'BTLG11', 'HCTR11', 'HGLG11', 'KNRI11', 'MXRF11',
       'RBVA11', 'VGHF11', 'VISC11', 'XPLG11', 'XPML11'], dtype=object)

In [191]:

fd_ticker = fd.get_papel('HCTR11')
print(fd_ticker)

2025-07-24 11:21:52,214 [detalhes.get_papel] INFO: detalhes: call: get..._papel()


None


In [157]:
df_fii['Valor da Operação'].sum()

np.float64(44735.06)

In [158]:
df_teste_valor_fii = df_fii[df_fii['Ticker'].isin(['BCFF11','BTHF11'])]
df_teste_valor_fii['Valor da Operação'].sum()

np.float64(9197.33)

In [159]:
df_teste_valor_fii.shape

(11, 11)

In [160]:
df_teste_valor_fii['Movimentação'].unique()

array(['Transferência - Liquidação'], dtype=object)

# Testando a carga da planilha com todos os dados

In [186]:
df_tudo = pd.read_excel(path_planilha_b3_tudo, dtype=str)
df_tudo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Entrada/Saída      363 non-null    object
 1   Data               363 non-null    object
 2   Movimentação       363 non-null    object
 3   Produto            363 non-null    object
 4   Instituição        363 non-null    object
 5   Quantidade         363 non-null    object
 6   Preço unitário     363 non-null    object
 7   Valor da Operação  363 non-null    object
dtypes: object(8)
memory usage: 22.8+ KB


c:\Projetos\pynvest\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [188]:
df_tudo[df_tudo['Produto'].str.contains('CDB')]['Produto'].unique()

array([], dtype=object)

In [162]:
df_tudo = preprocessar_dados_b3(df_tudo)
df_tudo = df_tudo.sort_values(by=['Ticker'], ascending=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Entrada/Saída      363 non-null    object 
 1   Data               363 non-null    object 
 2   Movimentação       363 non-null    object 
 3   Ticker             363 non-null    object 
 4   Descrição          363 non-null    object 
 5   Instituição        363 non-null    object 
 6   Quantidade         363 non-null    float64
 7   Preço unitário     363 non-null    float64
 8   Valor da Operação  363 non-null    float64
 9   Tipo de Ativo      363 non-null    object 
 10  Compra/Venda       363 non-null    bool   
dtypes: bool(1), float64(3), object(7)
memory usage: 28.8+ KB


In [163]:
df_tudo['Ticker'].unique()

array(['BBAS3', 'BCFF11', 'BCFF12', 'BIDI4', 'BTHF11', 'BTLG11', 'CVCB1',
       'CVCB3', 'HCTR11', 'HCTR12', 'HGLG11', 'ITSA4', 'ITUB4', 'KNRI11',
       'LWSA3', 'MGLU3', 'MXRF11', 'MXRF12', 'OIBR3', 'PETR4', 'PGMN3',
       'RBVA11', 'VGHF11', 'VGHF12', 'VISC11', 'VVAR3', 'XPLG11',
       'XPML11', 'XPML12'], dtype=object)

In [164]:
df_tudo = ajustar_movimentacoes_por_eventos(df_tudo, df_fusoes)
df_tudo['Ticker'].unique()

array(['BBAS3', 'BTHF11', 'BCFF12', 'BIDI4', 'BTLG11', 'CVCB1', 'CVCB3',
       'HCTR11', 'HCTR12', 'HGLG11', 'ITSA4', 'ITUB4', 'KNRI11', 'LWSA3',
       'MGLU3', 'MXRF11', 'MXRF12', 'OIBR3', 'PETR4', 'PGMN3', 'RBVA11',
       'VGHF11', 'VGHF12', 'VISC11', 'VVAR3', 'XPLG11', 'XPML11',
       'XPML12'], dtype=object)

In [165]:
df_tudo['Valor da Operação'].sum()

np.float64(53352.06999999999)

In [175]:
df_tudo['Movimentação'].unique()

array(['Transferência - Liquidação', 'Rendimento',
       'Juros Sobre Capital Próprio', 'Dividendo', 'Atualização',
       'Resgate', 'Cessão de Direitos', 'Cessão de Direitos - Solicitada',
       'Direito de Subscrição', 'Direitos de Subscrição - Não Exercido',
       'Desdobro', 'Transferência', 'Fração em Ativos',
       'Leilão de Fração'], dtype=object)

## Filtrando FII na planilha completa

In [166]:
def filtrar_fii(df: pd.DataFrame) -> pd.DataFrame:

    colunas_necessarias = ['Tipo de Ativo', 'Compra/Venda']
    if not all(col in df.columns for col in colunas_necessarias):
        return df

    filtro = (df['Tipo de Ativo'] == 'FII') & (df['Compra/Venda'])
    df_filtrado = df[filtro]
    return df_filtrado

In [167]:
df_tudo_filtrado = filtrar_fii(df_tudo)

## Validando o Resultado

In [180]:
df_tudo[df_tudo['Ticker'].str.endswith('11')]['Ticker'].unique()

array(['BTHF11', 'BTLG11', 'HCTR11', 'HGLG11', 'KNRI11', 'MXRF11',
       'RBVA11', 'VGHF11', 'VISC11', 'XPLG11', 'XPML11'], dtype=object)

In [181]:
df_tudo[df_tudo['Ticker'].str.endswith(('3', '4', '5', '6'))]['Ticker'].unique()

array(['BBAS3', 'BIDI4', 'CVCB3', 'ITSA4', 'ITUB4', 'LWSA3', 'MGLU3',
       'OIBR3', 'PETR4', 'PGMN3', 'VVAR3'], dtype=object)

array([], dtype=object)

In [168]:
df_tudo_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 19 to 343
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Entrada/Saída      60 non-null     object 
 1   Data               60 non-null     object 
 2   Movimentação       60 non-null     object 
 3   Ticker             60 non-null     object 
 4   Descrição          60 non-null     object 
 5   Instituição        60 non-null     object 
 6   Quantidade         60 non-null     float64
 7   Preço unitário     60 non-null     float64
 8   Valor da Operação  60 non-null     float64
 9   Tipo de Ativo      60 non-null     object 
 10  Compra/Venda       60 non-null     bool   
dtypes: bool(1), float64(3), object(7)
memory usage: 5.2+ KB


In [169]:
df_tudo_filtrado['Ticker'].unique()

array(['BTHF11', 'BTLG11', 'HCTR11', 'HGLG11', 'KNRI11', 'MXRF11',
       'RBVA11', 'VGHF11', 'VISC11', 'XPLG11', 'XPML11'], dtype=object)

In [170]:
df_tudo_filtrado['Valor da Operação'].sum()

np.float64(44735.06)

In [171]:
df_tudo_filtrado[['Ticker','Descrição','Tipo de Ativo']].drop_duplicates()


,Ticker,Descrição,Tipo de Ativo
19,BTHF11,FDO INV IMOB - FII BTG PACTUAL FUNDO DE FUNDOS,FII
32,BTHF11,FII - FII BTG PACTUAL FUNDO DE FUNDOS RESPONS ...,FII
77,BTLG11,BTG PACTUAL LOGISTICA FUNDO DE INVESTIMENTO IM...,FII
88,HCTR11,HECTARE CE - FDO INV IMOB,FII
89,HCTR11,HECTARE CE - FUNDO DE INVESTIMENTO IMOBILIÁRIO,FII
92,HGLG11,CSHG LOGÍSTICA FDO INV IMOB - FII,FII
136,KNRI11,KINEA RENDA IMOBILIÁRIA FDO INV IMOB - FII,FII
153,MXRF11,MAXI RENDA FUNDO DE INVESTIMENTO IMOBILIARIO -...,FII
170,MXRF11,MAXI RENDA FDO INV IMOB - FII,FII
242,RBVA11,FDO INV IMOB RIO BRAVO RENDA VAREJO - FII,FII


In [172]:
df_teste_valor_tudo_filtrado = df_tudo_filtrado[df_tudo_filtrado['Ticker'].isin(['BCFF11','BTHF11'])]
df_teste_valor_tudo_filtrado['Valor da Operação'].sum()

np.float64(9197.33)

In [173]:
df_teste_valor_tudo_filtrado.shape

(11, 11)

In [174]:
df_teste_valor_tudo_filtrado['Movimentação'].unique()

array(['Transferência - Liquidação'], dtype=object)